# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - NxNxN cube</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Using repo from dwalton76: https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [28]:
import zipfile
import sqlite3

import pandas as pd
import numpy as np

import os

os.chdir('rubiks-cube-NxNxN-solver')
# Print the current working directory
print("Current Working Directory: ", os.getcwd())

database_file = '../../solutions.db'
solution_method = "dwalton76 NxNxN algo"

FileNotFoundError: [Errno 2] No such file or directory: 'rubiks-cube-NxNxN-solver'

<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [29]:
with zipfile.ZipFile('../../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col='id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div>
 

In [30]:
U = ['U', 'F', 'R', 'B', 'L', 'D']
U_dict = {
    'A': 'U',
    'B': 'F',
    'C': 'R',
    'D': 'B',
    'E': 'L',
    'F': 'D',
}
def state2ubl(state):
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]


<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [31]:
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()


outputs = {}
for id, row in puzzles.iterrows():
    if row['puzzle_type'][:4] == 'cube':
        
        dim = int(row['puzzle_type'].split('/')[-1])
        if id not in range(205, 210):
            continue
        print(id) 
        moves = eval(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        for move in list(moves):
            moves['-'+move] = np.argsort(moves[move]).tolist()
        
        M = {}
        M["U"] = f'-d{dim-1}'
        M["R"] = "r0"
        M["B"] = f"-f{dim-1}"
        M["F"] = "f0"
        M["L"] = f"-r{dim-1}"
        M["D"] = "d0"
        
        if dim > 3:
            M["Uw"] = f'-d{dim-2}.-d{dim-1}'
            M["Rw"] = f"r0.r1"
            M["Bw"] = f'-f{dim-2}.-f{dim-1}'
            M["Fw"] = f"f0.f1"
            M["Lw"] = f'-r{dim-2}.-r{dim-1}'
            M["Dw"] = f"d0.d1"
        if dim >= 6:
            M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
            M["2Rw"] = f"r0.r1"
            M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
            M["2Fw"] = f"f0.f1"
            M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
            M["2Dw"] = f"d0.d1"
            
            width_max = dim // 2
            for i in range(3, width_max + 1):
                M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
                M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
                M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
                M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
                M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
                M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'
    
            
        for m in list(M):
            M[m+"2"] = M[m] + '.' + M[m]
            if "-" in M[m]:
                M[m+"'"] = M[m].replace("-","")
            else:
                M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])

        init_state = row['initial_state']
        sol_state = row['solution_state']


        if sol_state[0] == 'N':
            state = state2ubl(init_state)
            print(f'Starting {id}')
            output = !./rubiks-cube-solver.py --state $state
            outputs[id] = output
            if output[-1][:9] == 'Solution:':
                print(output[-1])
                sol = output[-1].split(': ')[1]
            else:
                for n in range(1, 21):
                    if 'Solution:' in output[-n]:
                        sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                        break
                
            # print(sol)
            mmoves = '.'.join([M[m] for m in sol.split(' ')])
            
            new_state = init_state
            for move in mmoves.split('.'):
                new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
            print(new_state)
            
            # I = ['r0.r1.r2','d0.d1.d2','f0.f1.f2']
            I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
            manipulations = [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for
                                                                              i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
            for init_moves in manipulations:
                temp_state = new_state
                if len(init_moves) > 0:
                    for move in init_moves.split('.'):
                        temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
                if temp_state == sol_state:
                    print(f'solved id: {id}')
                    if len(init_moves) > 0:
                        mmoves += '.' + init_moves
                    break
                    
            # validation
            state = row['initial_state'].split(";")
            for move_name in mmoves.split('.'):
                state = [state[i] for i in moves[move_name]]

            try:
                assert row['solution_state'].split(";") == state
                
                mmoves_length = len(mmoves.split('.'))
                
                select_query = "SELECT count FROM solutions WHERE id = ?"
    
                # Execute the query
                cursor.execute(select_query, (id,))
                response = cursor.fetchone()
                best_moves_length = response[0]
                
                if mmoves_length < best_moves_length:
                    # Insert the moves into the database
                    insert_query = ("INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)")
                    cursor.execute(insert_query, (id, mmoves, mmoves_length, solution_method))
                    conn.commit()
            except AssertionError:
                print(f"assertion error for {id}")
                print(row['solution_state'])
                print(';'.join(state))
                break

# Commit the changes and close the connection
conn.commit()
conn.close()


205


KeyError: 'N3'

In [ ]:
# import importlib
# import brute_force
# importlib.reload(brute_force)
# 
# puzzle = puzzles.loc[id]
# sol = brute_force.solve(puzzle, puzzle_info, ';'.join(state))
# sol